In [8]:
import pandas as pd
import random #minwise
import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np
import csv
from collections import Counter

In [9]:
class MinCount:
    
    def __init__(self,reservoirSize,debug_mode):
        self.debug = debug_mode
        self.reservoir_size = reservoirSize
        self.rns = []
        self.elements = [] 
    
    def try_add_candidate(self,candidate):
        if(self.debug):
            print("\n")
            print("Adding ",candidate)
        random_number = random.random()
        if(self.debug):
            print("Random Number: ",random_number)
        
        #if reservoir size < than reservoir max size always add
        if (len(self.elements) < self.reservoir_size):
            if(self.debug):
                print("Still have Space")
                print("CANDIDATE ADDED")
            self.rns.append(random_number)
            self.elements.append(candidate)
            return
        
        #else replace element with biggest rn
        else:
            if(self.debug):
                print("Try Substitute")
                print("Max randomnumber: ",max(self.rns)," in index ",self.rns.index(max(self.rns)))
            #if random number bigger than bigger random number dont add
            if random_number > max(self.rns):
                if(self.debug):
                    print("Random Number Higher than Max value in random numbers list")
                    print("CANDIDATE NOT ADDED")
                return
            if(self.debug):
                print("Random Number Lower than Max value in random numbers list")
                print("Added ",candidate," with rn: ",random_number)
                print("CANDIDATE ADDED WITH REPLACE")
            highest_rn_index = self.rns.index(max(self.rns))
            self.rns[highest_rn_index] = random_number
            self.elements[highest_rn_index] = candidate
            return
            
    def get_elements(self):
        return self.elements
    def get_rns(self):
        return self.rns
    
    def get_rns_and_elements(self):
        return list(zip(self.rns, self.elements))

In [26]:
#File_Paths
CTU_NETFLOW_FILEPATH = "./raw dataset/capture20110812.pcap.netflow.labeled"
CTU_NETFLOW_52_FILEPATH = "./capture20110818.pcap.netflow.labeled"
CSV_NETFLOW_FILEPATH = "./csv dataset/capture20110812.csv"
COLUMNS = ["Date flow start","Durat","Prot","Src IP Addr","Dst IP Addr","Flags","Tos","Packets Bytes","Flows","Label","Labels"]
HOSTS = ["147.32.84.165","147.32.84.170","147.32.84.134","147.32.84.164","147.32.87.36","147.32.80.9","147.32.87.11"]
fieldnames= ["Date flow start","Durat","Prot","Src IP Addr","Src Port","Dst IP Addr","Dst Port", "Flags","Tos","Packets Bytes","Flows","Label","Labels"]

In [20]:
packet_dict = {}


with open(CTU_NETFLOW_FILEPATH,"r") as netflow:
    with open(CSV_NETFLOW_FILEPATH,"w") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i,packet  in enumerate(netflow):
            if i ==0:
                continue
            if i %100000==0:
                print(i)
            packet = packet.rstrip('\n')
            split_packet = packet.split("\t")
            if "->" in split_packet:
                split_packet.remove("->")
            if "" in split_packet:
                split_packet.remove("")
            
            split_packet = [e for e in split_packet if e]
            
            packet_dict["Date flow start"] = split_packet[0]
            packet_dict["Durat"] = split_packet[1]
            packet_dict["Prot"] = split_packet[2]
            if ":" in split_packet[3]: 
                packet_dict["Src IP Addr"] = split_packet[3].split(":")[0]
                packet_dict["Src Port"] = split_packet[3].split(":")[1]
            else:
                packet_dict["Src IP Addr"] = split_packet[3]
                packet_dict.pop("Src Port", None)
            if ":" in split_packet[4]: 
                packet_dict["Dst IP Addr"] = split_packet[4].split(":")[0]
                packet_dict["Dst Port"] = split_packet[4].split(":")[1]  
            else:
                packet_dict["Dst IP Addr"] = split_packet[4]
                packet_dict.pop("Dst Port", None)
            
            packet_dict["Flags"] = split_packet[5]
            packet_dict["Tos"] = split_packet[6]
            packet_dict["Packets Bytes"] = split_packet[7]
            packet_dict["Flows"] = split_packet[8]
            packet_dict["Label"] = split_packet[9]
            packet_dict["Labels"] = split_packet[10]
            writer.writerow(packet_dict)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

# Parse File

In [25]:
#Open File
amount_of_packets = 10000
packets =[]
minCounter = MinCount(1000,False)#DO NOT TURN DEBUG MODE FOR ALL THE DATA
with open(CTU_NETFLOW_FILEPATH,"r") as netflow:
    #File is an Argus Unidirectional file created with ra -r so not rly csv. 
    #Let's parse by hand.
    for i,packet  in enumerate(netflow):
        #skip header
        if i == 0:
            continue
            
        packet = packet.rstrip('\n')
        split_packet = packet.split("\t")
        if i%100000==0:
            print("Packet ",i)
        
        if "->" in split_packet:
            split_packet.remove("->")
        if "" in split_packet:
            split_packet.remove("")
        #print(split_packet[3])
        if ":" in split_packet[3]: 
            split_packet[3]= split_packet[3].split(":")[0]
        #print(split_packet[4])
        if ":" in split_packet[4]:
            split_packet[4]= split_packet[4].split(":")[0]
        #print(split_packet[3]," -> ",split_packet[4])
        
        split_packet = [e for e in split_packet if e]
        
        #minCounter.try_add_candidate(split_packet[3])#Source IP
        if split_packet[4] not in HOSTS:
            minCounter.try_add_candidate(split_packet[4])#Dest IP
        if split_packet[3] not in HOSTS:
            minCounter.try_add_candidate(split_packet[3])
        #else:
            #print(split_packet[4]," is host")
        #print(split_packet)
        #packets.append(split_packet)
        
        """if i == amount_of_packets:
            break"""
        

Packet  100000
Packet  200000
Packet  300000
Packet  400000
Packet  500000
Packet  600000
Packet  700000
Packet  800000
Packet  900000
Packet  1000000
Packet  1100000
Packet  1200000
Packet  1300000
Packet  1400000
Packet  1500000
Packet  1600000
Packet  1700000
Packet  1800000
Packet  1900000
Packet  2000000
Packet  2100000
Packet  2200000
Packet  2300000
Packet  2400000
Packet  2500000
Packet  2600000
Packet  2700000
Packet  2800000
Packet  2900000
Packet  3000000
Packet  3100000
Packet  3200000
Packet  3300000
Packet  3400000
Packet  3500000
Packet  3600000
Packet  3700000
Packet  3800000
Packet  3900000
Packet  4000000
Packet  4100000
Packet  4200000
Packet  4300000
Packet  4400000
Packet  4500000
Packet  4600000
Packet  4700000
Packet  4800000
Packet  4900000
Packet  5000000
Packet  5100000
Packet  5200000
Packet  5300000
Packet  5400000
Packet  5500000
Packet  5600000
Packet  5700000
Packet  5800000
Packet  5900000
Packet  6000000
Packet  6100000
Packet  6200000
Packet  6300000
P

In [24]:
#Open File
counter_b = 0
counter_l = 0
counter_bot = 0
with open(CTU_NETFLOW_FILEPATH,"r") as netflow:
    #File is an Argus Unidirectional file created with ra -r so not rly csv. 
    #Let's parse by hand.
    for i,packet  in enumerate(netflow):
        if i%1000000==0:
            print("Packet ",i)
        packet = packet.rstrip('\n')
        split_packet = packet.split("\t")
        if (split_packet[-1]=="Background"):
            counter_b+=1
        if (split_packet[-1]=="Botnet"):
            counter_bot+=1
        if (split_packet[-1]=="LEGITIMATE"):
            counter_l+=1
print("Background: ", counter_b)
print("Legitimate: ",counter_l)
print("Botnet: ",counter_bot)
print("------")
print("Total: ", counter_bot+counter_l+counter_b)

Packet  0
Packet  100000
Packet  200000
Packet  300000
Packet  400000
Packet  500000
Packet  600000
Packet  700000
Packet  800000
Packet  900000
Packet  1000000
Packet  1100000
Packet  1200000
Packet  1300000
Packet  1400000
Packet  1500000
Packet  1600000
Packet  1700000
Packet  1800000
Packet  1900000
Packet  2000000
Packet  2100000
Packet  2200000
Packet  2300000
Packet  2400000
Packet  2500000
Packet  2600000
Packet  2700000
Packet  2800000
Packet  2900000
Packet  3000000
Packet  3100000
Packet  3200000
Packet  3300000
Packet  3400000
Packet  3500000
Packet  3600000
Packet  3700000
Packet  3800000
Packet  3900000
Packet  4000000
Packet  4100000
Packet  4200000
Packet  4300000
Packet  4400000
Packet  4500000
Packet  4600000
Packet  4700000
Packet  4800000
Packet  4900000
Packet  5000000
Packet  5100000
Packet  5200000
Packet  5300000
Packet  5400000
Packet  5500000
Packet  5600000
Packet  5700000
Packet  5800000
Packet  5900000
Packet  6000000
Packet  6100000
Packet  6200000
Packet 

In [2]:
df = pd.DataFrame(packets,columns=COLUMNS)

NameError: name 'pd' is not defined

In [6]:
minCounter.get_rns_and_elements()

[(1.1220165886505207e-05, '79.177.123.143'),
 (6.445429730073116e-05, '147.32.87.27'),
 (6.91874823576244e-05, '147.32.84.138'),
 (3.0915184347946045e-05, '68.196.248.158'),
 (2.2974833725819543e-05, '147.32.85.25'),
 (5.871617344532787e-05, '147.32.85.25'),
 (5.8596023807200304e-05, '147.32.85.21'),
 (6.507082397433361e-05, '189.100.253.44'),
 (6.122271097241949e-05, '147.32.84.229'),
 (3.9150108006036355e-05, '147.32.85.25'),
 (7.218893792815262e-05, '147.32.84.2'),
 (6.94341409367416e-05, '182.252.138.47'),
 (5.512961737863531e-05, '147.32.85.25'),
 (6.445706747404412e-05, '197.1.63.0'),
 (5.651517135796258e-05, '66.249.72.50'),
 (6.789212105495768e-05, '115.78.130.8'),
 (5.281166722836961e-05, '82.208.56.90'),
 (3.899812682783832e-07, '147.32.87.38'),
 (1.6289281748682605e-05, '94.100.187.196'),
 (2.0924619278051004e-05, '147.32.85.25'),
 (1.987649962398308e-07, '149.13.32.147'),
 (2.6131042498156454e-05, '147.32.85.25'),
 (1.87283789838677e-05, '147.32.84.50'),
 (4.215087134595841

In [8]:
counter = Counter(minCounter.get_elements())

print(len(counter))
print(counter.most_common()[:10])

455
[('147.32.84.229', 149), ('147.32.84.138', 53), ('147.32.85.25', 45), ('147.32.85.100', 25), ('147.32.85.34', 25), ('147.32.84.59', 20), ('147.32.86.20', 19), ('188.138.84.239', 16), ('147.32.84.2', 14), ('147.32.80.13', 11)]


In [15]:
with open("counter_min_wise_sampling.pkl","wb") as counter_file_min_wise:
    pkl.dump(counter,counter_file_min_wise)

In [6]:
with open("counter_min_wise_sampling.pkl","rb") as counter_file_min_wise:
    counter = pkl.load(counter_file_min_wise)

In [8]:
counter.most_common()

[('147.32.84.229', 149),
 ('147.32.84.138', 53),
 ('147.32.85.25', 45),
 ('147.32.85.34', 25),
 ('147.32.85.100', 25),
 ('147.32.84.59', 20),
 ('147.32.86.20', 19),
 ('188.138.84.239', 16),
 ('147.32.84.2', 14),
 ('147.32.80.13', 11),
 ('147.32.84.171', 9),
 ('147.32.86.190', 9),
 ('87.98.230.229', 8),
 ('147.32.87.27', 7),
 ('147.32.84.131', 6),
 ('147.32.85.56', 6),
 ('147.32.84.130', 5),
 ('147.32.85.74', 5),
 ('147.32.85.5', 5),
 ('147.32.84.166', 5),
 ('147.32.87.78', 5),
 ('182.252.138.47', 5),
 ('147.32.86.89', 4),
 ('239.255.255.250', 4),
 ('147.32.203.229', 4),
 ('157.55.185.43', 4),
 ('147.32.85.21', 4),
 ('Broadcast', 4),
 ('65.52.205.145', 4),
 ('209.85.148.105', 3),
 ('147.32.84.36', 3),
 ('217.29.8.4', 3),
 ('147.32.86.230', 3),
 ('85.17.234.14', 3),
 ('70.37.160.39', 3),
 ('74.125.232.216', 3),
 ('147.32.84.19', 3),
 ('147.32.85.84', 3),
 ('147.32.84.7', 3),
 ('74.125.39.125', 3),
 ('147.32.84.188', 3),
 ('147.32.84.20', 3),
 ('209.85.148.103', 3),
 ('147.32.84.184', 3),


# INFECTED HOST TRIAL 

In [38]:
infected_host={"saruman": "147.32.84.165"}

In [40]:
#Open File
going_packets =[]
comming_packets = []
with open(CTU_NETFLOW_52_FILEPATH,"r") as netflow:
    #File is an Argus Unidirectional file created with ra -r so not rly csv. 
    #Let's parse by hand.
    for i,packet  in enumerate(netflow):
        #skip header
        if i == 0:
            continue
            
        packet = packet.rstrip('\n')
        split_packet = packet.split("\t")
        
        if len(split_packet)==1:
            split_packet = packet.split(" ")
            split_packet[0] = split_packet[0]+" "+ split_packet[1]
            del split_packet[1]
        
        if i%100000==0:
            print("Packet ",i)
        
        if "->" in split_packet:
            split_packet.remove("->")
            
        if "" in split_packet:
            split_packet.remove("")
        
        if ":" in split_packet[3]: 
            split_packet[3]= split_packet[3].split(":")[0]
        
        if ":" in split_packet[4]:
            split_packet[4]= split_packet[4].split(":")[0]
        
        split_packet = [e for e in split_packet if e]
        if split_packet[-1]!="Background":
            if split_packet[4]==infected_host['saruman']:
                comming_packets.append(split_packet)
            if split_packet[3]==infected_host['saruman']:
                going_packets.append(split_packet)
            
        """if i == amount_of_packets:
            break"""
        

Packet  100000
Packet  200000
Packet  300000
Packet  400000
Packet  500000
Packet  600000
Packet  700000
Packet  800000
Packet  900000
Packet  1000000
Packet  1100000
Packet  1200000
Packet  1300000
Packet  1400000
Packet  1500000
Packet  1600000
Packet  1700000
Packet  1800000
Packet  1900000
Packet  2000000
Packet  2100000
Packet  2200000
Packet  2300000
Packet  2400000
Packet  2500000
Packet  2600000
Packet  2700000
Packet  2800000
Packet  2900000
Packet  3000000
Packet  3100000
Packet  3200000
Packet  3300000
Packet  3400000
Packet  3500000
Packet  3600000
Packet  3700000
Packet  3800000
Packet  3900000
Packet  4000000
Packet  4100000
Packet  4200000
Packet  4300000
Packet  4400000
Packet  4500000
Packet  4600000
Packet  4700000
Packet  4800000
Packet  4900000
Packet  5000000
Packet  5100000


In [42]:
going_protocols=set()
comming_protocols=set()
for x in going_packets:
    going_protocols.add(x[2])
for x in comming_packets:
    comming_protocols.add(x[2])
print(going_protocols)
print(comming_protocols)

{'ICMP', 'TCP', 'UDP'}
{'ICMP', 'TCP', 'UDP'}


In [45]:
going_flags = set()
comming_flags = set()
for x in going_packets:
    going_flags.add(x[2]+"-"+x[5])
for x in comming_packets:
    comming_flags.add(x[2]+"-"+x[5])
print("GOING")
print(going_flags)
print("COMMING")
print(comming_flags)

GOING
{'ICMP-UR', 'UDP-INT', 'ICMP-URHU', 'ICMP-NRA', 'TCP-PA_', 'ICMP-DNQ', 'ICMP-TXD', 'ICMP-IAH', 'ICMP-RED', 'ICMP-SEC', 'ICMP-SRC', 'ICMP-TRC', 'ICMP-IRQ', 'ICMP-___', 'ICMP-NNS', 'ICMP-RTA', 'TCP-SA_', 'ICMP-MHR', 'TCP-SPA_', 'ICMP-TST', 'ICMP-ECO', 'ICMP-NNA', 'ICMP-RTS', 'ICMP-DCE', 'ICMP-NRS', 'ICMP-MRQ', 'ICMP-UNK', 'ICMP-MRP', 'ICMP-DNP', 'TCP-A_', 'ICMP-WAY', 'ICMP-PHO', 'ICMP-PTB', 'ICMP-SKP', 'TCP-SRPA_', 'ICMP-ROB', 'ICMP-AHA', 'ICMP-MAS', 'TCP-RA_', 'ICMP-PAR'}
COMMING
{'ICMP-MSR', 'ICMP-IRR', 'UDP-INT', 'ICMP-ECR', 'TCP-FSPA_', 'TCP-SPA_', 'TCP-PA_', 'ICMP-ECO', 'TCP-A_', 'ICMP-TSR', 'TCP-S_'}


In [ ]:
def calculate_protocol_flag(netflow):
    return (netflow[2]+"-"+netflow[5])

In [49]:
goingFlagsDict=zip(going_flags,range(len(going_flags)))

In [50]:
list(goingFlagsDict)

[('ICMP-UR', 0),
 ('UDP-INT', 1),
 ('ICMP-URHU', 2),
 ('ICMP-NRA', 3),
 ('TCP-PA_', 4),
 ('ICMP-DNQ', 5),
 ('ICMP-TXD', 6),
 ('ICMP-IAH', 7),
 ('ICMP-RED', 8),
 ('ICMP-SEC', 9),
 ('ICMP-SRC', 10),
 ('ICMP-TRC', 11),
 ('ICMP-IRQ', 12),
 ('ICMP-___', 13),
 ('ICMP-NNS', 14),
 ('ICMP-RTA', 15),
 ('TCP-SA_', 16),
 ('ICMP-MHR', 17),
 ('TCP-SPA_', 18),
 ('ICMP-TST', 19),
 ('ICMP-ECO', 20),
 ('ICMP-NNA', 21),
 ('ICMP-RTS', 22),
 ('ICMP-DCE', 23),
 ('ICMP-NRS', 24),
 ('ICMP-MRQ', 25),
 ('ICMP-UNK', 26),
 ('ICMP-MRP', 27),
 ('ICMP-DNP', 28),
 ('TCP-A_', 29),
 ('ICMP-WAY', 30),
 ('ICMP-PHO', 31),
 ('ICMP-PTB', 32),
 ('ICMP-SKP', 33),
 ('TCP-SRPA_', 34),
 ('ICMP-ROB', 35),
 ('ICMP-AHA', 36),
 ('ICMP-MAS', 37),
 ('TCP-RA_', 38),
 ('ICMP-PAR', 39)]

In [51]:
commingFlagsDict=zip(comming_flags,range(len(comming_flags)))

In [53]:
list(commingFlagsDict)

[('ICMP-MSR', 0),
 ('ICMP-IRR', 1),
 ('UDP-INT', 2),
 ('ICMP-ECR', 3),
 ('TCP-FSPA_', 4),
 ('TCP-SPA_', 5),
 ('TCP-PA_', 6),
 ('ICMP-ECO', 7),
 ('TCP-A_', 8),
 ('ICMP-TSR', 9),
 ('TCP-S_', 10)]